In [54]:
import torch
import torch.nn as nn
from torch import optim
#from torchdiffeq import odeint_adjoint as odeint
from scipy.linalg import expm
from torchdiffeq import odeint
import numpy as np

In [61]:
def dydt(y, t, A):
    return torch.mm(y,A)

In [81]:
def phi_A(y,A):
    return odeint(lambda t,x : dydt(x,t,A), y, torch.tensor([0., 1.]))[1]

In [3]:
class ODEFunc(torch.nn.Module):
    def __init__(self, A):
        super(ODEFunc, self).__init__()
        self.A = torch.nn.Parameter(torch.tensor(A))
        
    def forward(self, t, y):
        return dydt(y, t, self.A)

In [104]:
class NeuralODE(torch.nn.Module):
    def __init__(self, A_init):
        super(NeuralODE, self).__init__()
        self.func = ODEFunc(A_init)
        self.hidden_layer = torch.nn.Linear(2, 10)
        self.output_layer = torch.nn.Linear(10, 4)
        
    def forward(self, y):
        y = self.hidden_layer(y)
        y = torch.relu(y)
        y = self.output_layer(y)
        return y
    
    def get_A(self):
        return self.func.A

In [148]:
A_true = torch.tensor([[1., 2.], [3., 4.]])
training_losses={}
frob_losses={}

In [152]:
def train_model(model, x_data,y_data,epochs=500, lr=0.01):
    training_loss=[]
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = odeint(model.func, x_data, torch.tensor([0., 1.]), method='dopri5')
        loss = criterion(y_pred, y_data)
        loss.backward()
        optimizer.step()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Training Loss: {loss:.4f}")
            print(neural_ode.get_A())
            training_loss.append(int(loss.detach().numpy().item()))
    return training_loss

In [153]:
n_samples=100

x_data = torch.randn(n_samples, 2)
y_data = phi_A(x_data,A_true)

In [154]:
neural_ode = NeuralODE(torch.eye(2))

training_loss=train_model(neural_ode, x_data,y_data)
A_estimated = neural_ode.get_A()

#ajout des résultats pour chaque n_samples
training_losses[n_samples]=training_loss
frob_losses[n_samples]=np.linalg.norm((A_true-A_estimated).detach().numpy())

print(f"Estimated A:\n{A_estimated}")

C:\Users\user\AppData\Local\Temp\ipykernel_12628\3190449401.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.A = torch.nn.Parameter(torch.tensor(A))
C:\Users\user\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([100, 2])) that is different to the input size (torch.Size([2, 100, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Training Loss: 21579.8750
Parameter containing:
tensor([[1.0100, 0.0100],
        [0.0100, 1.0100]], requires_grad=True)
Epoch 20, Training Loss: 21457.0117
Parameter containing:
tensor([[1.2167, 0.2159],
        [0.2147, 1.2140]], requires_grad=True)
Epoch 40, Training Loss: 21249.0430
Parameter containing:
tensor([[1.4526, 0.4478],
        [0.4417, 1.4373]], requires_grad=True)
Epoch 60, Training Loss: 20861.2578
Parameter containing:
tensor([[1.7301, 0.7187],
        [0.7044, 1.6927]], requires_grad=True)
Epoch 80, Training Loss: 20088.9355
Parameter containing:
tensor([[2.0524, 1.0333],
        [1.0100, 1.9891]], requires_grad=True)
Epoch 100, Training Loss: 18516.3359
Parameter containing:
tensor([[2.4160, 1.3905],
        [1.3598, 2.3304]], requires_grad=True)
Epoch 120, Training Loss: 15634.7412
Parameter containing:
tensor([[2.8055, 1.7776],
        [1.7437, 2.7093]], requires_grad=True)
Epoch 140, Training Loss: 12865.5820
Parameter containing:
tensor([[3.1228, 2.1219